In [2]:
import pandas as pd
import re
from tqdm import tqdm
import time
import numpy as np

Cresci Dataset

In [39]:
df = pd.read_csv('all_tweets_cresci.csv')

/var/folders/cn/hl5lj7gd17bft8_srrs_2zvr0000gn/T/ipykernel_64002/3289021857.py:1: DtypeWarning: Columns (10,29,30,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('all_tweets_cresci.csv')


In [42]:
## Data preprocessing
df['user_id'] = df['user_id'].astype(int).astype(str)
list = df['user_id'].unique()
df_users = pd.DataFrame(list)
df_users.rename(columns={0: 'user_id'}, inplace=True)
df_users = pd.merge(df_users, df[['name', 'screen_name', 'location', 'description', 'following', 'created_at_user', 'friends_count', 'lang', 'user_id', 'db']], on= 'user_id', how = 'left')
df_users = df_users.drop_duplicates(subset='user_id')


In [44]:
## creating tweet_counts for 15_min interval
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['rounded_15min'] = df['timestamp'].dt.floor('15min')
tweet_counts = df.groupby(['user_id', 'rounded_15min']).size().reset_index(name='tweet_count')

In [46]:
##15_min_summary
user_summary_15min = tweet_counts.groupby('user_id')['tweet_count'].agg(['sum','max', 'min']).reset_index()
user_summary_15min.rename(columns={'sum': 'total_tweets_15min','max': 'max_tweets_15min', 'min': 'min_tweets_15min'}, inplace=True)
df_users = df_users.merge(user_summary_15min, on='user_id', how='left')

In [47]:
### mean number of words in tweet
df['text'] = df['text'].apply(lambda x: str(x) if isinstance(x, (str, float)) else '')
df['word_count'] = df['text'].str.split().apply(len)
ser_mean_word_count = df.groupby('user_id')['word_count'].agg(['mean','std']).reset_index()
ser_mean_word_count.rename(columns={'mean': 'mean_word_count','std':'std_word_count'}, inplace=True)
df_users = df_users.merge(ser_mean_word_count, on='user_id', how='left')

In [50]:
### retweeted count
retweeted_count = df.groupby('user_id')['retweet_count'].agg(['mean', 'std']).reset_index()
retweeted_count.rename(columns={'mean': 'mean_retweet_count','std':'std_retweet_count'}, inplace=True)
df_users = df_users.merge(retweeted_count, on='user_id', how='left')

In [51]:
## Mentions
df['mentions_count'] = df['text'].str.count('@')
mentions_count = df.groupby('user_id')['mentions_count'].agg(['mean', 'std']).reset_index()
mentions_count.rename(columns={'mean': 'mean_mentions_count', 'std':'std_mentions_count'}, inplace=True)
df_users = df_users.merge(mentions_count, on='user_id', how='left')

In [52]:
## Hashtags
df['hashtags_counts'] = df['text'].str.count('#')
hashtag_count = df.groupby('user_id')['mentions_count'].agg(['mean', 'std']).reset_index()
hashtag_count.rename(columns={'mean': 'mean_hashtag_count', 'std':'std_hashtag_count'}, inplace=True)
df_users = df_users.merge(hashtag_count, on='user_id', how='left')

In [53]:
## URls
df['urls_counts'] = df['text'].str.count('http')
hashtag_count = df.groupby('user_id')['urls_counts'].agg(['mean', 'std']).reset_index()
hashtag_count.rename(columns={'mean': 'mean_urls_count', 'std':'std_urls_count'}, inplace=True)
df_users = df_users.merge(hashtag_count, on='user_id', how='left')


In [54]:
### types of tweets, genuine - real tweets, social_tweets1, social_tweets2 - spambots that look like real people, 'traditional_spambots1'
df_users['db'].unique()

array(['genuine', 'social_tweets1', 'social_tweets2',
       'traditional_spambots1'], dtype=object)

In [56]:
df_users.to_csv('users_with_variables.csv')

In [ ]:
###### problem with dataset: not a lot of genuine users 437 

In [74]:
len(df_users[df_users['db']=='genuine'])

437

Caverlee 2011

In [109]:
file_path = 'legitimate_users.txt'
df_legit_users = pd.read_csv(file_path, delimiter='\t', header=None)
df_legit_users.columns = ['userid','created_at','collected_at','following','followers','number_of_tweets','length_screen_name','length_description']

In [110]:
file_path = 'legitimate_users_tweets.txt'
df_legit_tweets = pd.read_csv(file_path, delimiter='\t', header=None)
df_legit_tweets.columns = ['userid','tweetid','text','tweet_created_at']

In [113]:
file_path = 'content_polluters.txt'
df_bots_users = pd.read_csv(file_path, delimiter='\t', header=None)
df_bots_users.columns = ['userid','created_at','collected_at','following','followers','number_of_tweets','length_screen_name','length_description']

In [114]:
file_path = 'content_polluters_tweets.txt'
df_bots_tweets = pd.read_csv(file_path, delimiter='\t', header=None)
df_bots_tweets.columns = ['userid','tweetid','text','tweet_created_at']

In [115]:
## creating labels
df_bots_tweets['bot']=1
df_bots_users['bot']=1
df_legit_tweets['bot']=0
df_legit_users['bot']=0

In [116]:
## merging into users and tweets df
users = pd.concat([df_bots_users, df_legit_users], axis = 0)
tweets = pd.concat([df_legit_tweets, df_bots_tweets], axis = 0)

In [117]:
## seems like two datasets do not concide perfectly 
users_new = users.merge(tweets[['userid']], on='userid')
users_new = users_new.drop_duplicates()
tweets_new = tweets.merge(users[['userid']], on='userid')
tweets_new = tweets_new.drop_duplicates()

In [118]:
print(len(users), len(users_new))
print(len(tweets), len(tweets_new))

41499 39897
5580068 5579692


In [119]:
### there are some users that were detected as bots and genuine users in different time, approx 50 of them, lets delete them
df = users_new.groupby('userid').size().reset_index(name='count')
df_to_delete = df[df['count']>1]


users_new = users_new.merge(df_to_delete[['userid']], on='userid', how='left', indicator=True)
users_new = users_new[users_new['_merge'] == 'left_only']
users_new.drop('_merge', axis=1, inplace=True)


tweets_new = tweets_new.merge(df_to_delete[['userid']], on='userid', how='left', indicator=True)
tweets_new = tweets_new[tweets_new['_merge'] == 'left_only']
tweets_new.drop('_merge', axis=1, inplace=True)

In [120]:
print(len(tweets_new['userid'].unique()), len(users_new['userid'].unique()))

39809 39809


In [121]:
### lets set to original name datasets
tweets = tweets_new
users = users_new

In [122]:
### Time variables
tweets['tweet_created_at'] = pd.to_datetime(tweets['tweet_created_at'])

tweets['rounded_15min'] = tweets['tweet_created_at'].dt.floor('15min')
tweet_counts = tweets.groupby(['userid', 'rounded_15min']).size().reset_index(name='tweet_count')

user_summary_15min = tweet_counts.groupby('userid')['tweet_count'].agg(['max', 'min', 'std', 'mean']).reset_index()
user_summary_15min.rename(columns={'max': 'max_tweets_15min', 'min': 'min_tweets_15min', 'std':'std_tweets_15_min', 'mean':'mean_tweets_15min'}, inplace=True)

users = users.merge(user_summary_15min, on='userid', how='left')

In [123]:
### mean number of words in tweet
tweets['text'] = tweets['text'].apply(lambda x: str(x) if isinstance(x, (str, float)) else '')
tweets['word_count'] = tweets['text'].str.split().apply(len)
ser_mean_word_count = tweets.groupby('userid')['word_count'].agg(['mean','std']).reset_index()
ser_mean_word_count.rename(columns={'mean': 'mean_word_count','std':'std_word_count'}, inplace=True)
users = users.merge(ser_mean_word_count, on='userid', how='left')

In [124]:
## Mentions
tweets['mentions_count'] = tweets['text'].str.count('@')
mentions_count = tweets.groupby('userid')['mentions_count'].agg(['mean', 'std']).reset_index()
mentions_count.rename(columns={'mean': 'mean_mentions_count', 'std':'std_mentions_count'}, inplace=True)
users = users.merge(mentions_count, on='userid', how='left')

In [125]:
## Hashtags
tweets['hashtags_counts'] = tweets['text'].str.count('#')
hashtag_count = tweets.groupby('userid')['mentions_count'].agg(['mean', 'std']).reset_index()
hashtag_count.rename(columns={'mean': 'mean_hashtag_count', 'std':'std_hashtag_count'}, inplace=True)
users = users.merge(hashtag_count, on='userid', how='left')

In [126]:
## URls
tweets['urls_counts'] = tweets['text'].str.count('http')
hashtag_count = tweets.groupby('userid')['urls_counts'].agg(['mean', 'std']).reset_index()
hashtag_count.rename(columns={'mean': 'mean_urls_count', 'std':'std_urls_count'}, inplace=True)
users = users.merge(hashtag_count, on='userid', how='left')

In [127]:
## followers to following ratio
users['followers_to_following'] = np.where(users['following'] == 0, np.nan, users['followers'] / users['following'])

In [129]:
users.to_csv('users_caverlee.csv', index = False)